# IRDL tutorial
### An Intermediate Representation Definition Language (IRDL) for SSA Compilers

## Imports and setup

In [1]:
# xDSL should be available in the environment.

from xdsl.ir import MLContext
from xdsl.dialects.arith import Arith
from xdsl.dialects.builtin import Builtin
from xdsl.printer import Printer

# MLContext, containing information about the registered dialects
context = MLContext()

# Some useful dialects
context.register_dialect(Arith)
context.register_dialect(Builtin)

# Printer used to pretty-print MLIR data structures
printer = Printer()

## Introduction

xDSL is an extensible compiler, meaning that new operations, attributes, and types can be added. xDSL provides an embedded DSL, IRDL, to define new dialects.
This tutorial aims to show the different features IRDL has, and presents examples on how to use them.

## Attribute constraints

Attribute constraints represent invariants over attributes, and are an important concept for defining new attributes and operations. In practice, an attribute constraint is a child class of `AttrConstraint` that implements a `verify` method. The method takes an attribute to verify as parameter, and a dictionary associating constraint variables to attributes. `verify` does not return anything, but raises an exception if the invariant is not respected.

### Any constraint

An `Any` constraint will never trigger an exception, and will always pass:

In [2]:
# NBVAL_CHECK_OUTPUT

from xdsl.irdl import AnyAttr
from xdsl.dialects.builtin import i64, StringAttr, IndexType, IntegerType

# Construct the constraint
any_constraint = AnyAttr()

# This will pass without triggering an exception
any_constraint.verify(i64, {})
any_constraint.verify(StringAttr("ga"), {})

### Equality constraint

An equality constraint ensures that the attribute is equal to one provided to the constraint:

In [3]:
from xdsl.irdl import EqAttrConstraint
from xdsl.dialects.builtin import i32

# Construct the constraint
eq_constraint = EqAttrConstraint(i64)

# This will pass without triggering an exception
eq_constraint.verify(i64, {})

# This will trigger an exception
try:
    eq_constraint.verify(i32, {})
except Exception as e:
    print(e)

Expected attribute i64 but got i32


### Base attribute constraint

A base attribute constraint ensures that the attribute base type is equal to an expected attribute base type:

In [4]:
from xdsl.irdl import BaseAttr
from xdsl.dialects.builtin import IntAttr
from xdsl.utils.exceptions import VerifyException

# Construct the constraint
base_constraint = BaseAttr(StringAttr)

# This will pass without triggering an exception
base_constraint.verify(StringAttr("ga"), {})
base_constraint.verify(StringAttr("bu"), {})

# This will trigger an exception
try:
    base_constraint.verify(i32, {})
except VerifyException as e:
    print(e)

i32 should be of base attribute string


In [5]:
# This too
try:
    base_constraint.verify(IntAttr(3), {})
except VerifyException as e:
    print(e)

#int<3> should be of base attribute string


### Attribute constraint coercion

To simplify the definitions of constraints, constraint constructors expecting an attribute constraints will coerce `Attribute` to an equality attribute constraint, and will coerce an `Attribute` type to a base attribute constraint. this is done using the `attr_constr_coercion` function:

In [6]:
# NBVAL_CHECK_OUTPUT
from xdsl.irdl import EqAttrConstraint, attr_constr_coercion, AnyOf
from xdsl.dialects.builtin import i32

assert attr_constr_coercion(i32) == EqAttrConstraint(i32)
assert attr_constr_coercion(StringAttr) == BaseAttr(StringAttr)
assert AnyOf([i32, StringAttr]) == AnyOf([EqAttrConstraint(i32), BaseAttr(StringAttr)])

### Or constraint

An or constraint ensures that one of the given attribute constraints is satisfied by the attribute:

In [7]:
# Construct the constraint. Note that we are using the coercion defined previously.
or_constraint = AnyOf([i32, StringAttr])

# This will pass without triggering an exception, since the first constraint is satisfied
or_constraint.verify(i32, {})

# This will pass without triggering an exception, since the second constraint is satisfied
or_constraint.verify(StringAttr("ga"), {})
or_constraint.verify(StringAttr("bu"), {})

# This will trigger an exception, since none of the constraints are satisfied
try:
    or_constraint.verify(i64, {})
except Exception as e:
    print(e)

Unexpected attribute i64


### Parametric attribute constraint

A parametric attribute constraint is satisfied by parametric attributes of a certain base type. In addition, parametric attribute constraints specify constraints for each of the parameters of the attribute:

In [8]:
from xdsl.dialects.builtin import IntegerAttr
from xdsl.irdl import ParamAttrConstraint

# Construct the constraint. Note that we are using the coercion defined previously.
param_constraint = ParamAttrConstraint(IntegerAttr, [IntAttr, i32])

# This will pass without triggering an exception.
param_constraint.verify(IntegerAttr(IntAttr(42), i32), {})
param_constraint.verify(IntegerAttr(IntAttr(23), i32), {})

# This will trigger an exception, since the attribute type is not the expected one
try:
    param_constraint.verify(i64, {})
except VerifyException as e:
    print(e)

i64 should be of base attribute integer


In [9]:
# This will trigger an exception, since the second parameter constraint is not satisfied

try:
    param_constraint.verify(IntegerAttr(IntAttr(42), i64), {})
except VerifyException as e:
    print(e)

Expected attribute i32 but got i64


### Constraint variables

Constraint variables are used to specify equality between attributes in operation and attribute definitions. They also contain a constraint that must be satisfied.
The first time a contraint variable is used, it will check that the constraint is satisfied. If it is satisfied, it sets the variable to the given attribute. If a constraint variable is already set, it will check that the given attribute is equal to the one already set. Two constraint variables with the same name are considered equal, and are expected to carry the same contraint.

In [10]:
from xdsl.ir import Attribute
from xdsl.irdl import VarConstraint

var_constraint = VarConstraint("T", BaseAttr(IntegerType))
constraint_variables: dict[str, Attribute] = {}

# The underlying constraint is not satisfied by the given attribute
try:
    var_constraint.verify(IndexType(), constraint_variables)
except VerifyException as e:
    print(e)

index should be of base attribute integer_type


In [11]:
# The constraint sets the constraint variable if the attribute verifies
var_constraint.verify(i32, constraint_variables)
print(constraint_variables["T"])

i32


In [12]:
# Since the variable is set, the constraint can now only be satisfied by the same attribute
try:
    var_constraint.verify(i64, constraint_variables)
except VerifyException as e:
    print(e)

attribute i32 expected from variable 'T', but got i64


### Custom constraints

Users can define their own constraints for their own types. For instance, here is the definition of the `ArrayOfConstraint` constraint, which expects an `ArrayAttr` where all elements of the array satisfy a specific constraint:

In [13]:
# NBVAL_CHECK_OUTPUT

from xdsl.ir import Attribute
from xdsl.dialects.builtin import ArrayAttr
from xdsl.irdl import AttrConstraint

from typing import Union, Type
from dataclasses import dataclass

from xdsl.utils.exceptions import VerifyException
from xdsl.utils.hints import isa


@dataclass
class ArrayOfConstraint(AttrConstraint):
    # The constraint that needs to be satisfied by all elements of the array
    elem_constr: AttrConstraint

    # The custom init applies the attribute constraint coercion
    def __init__(self, constr: Union[Attribute, Type[Attribute], AttrConstraint]):
        self.elem_constr = attr_constr_coercion(constr)

    # Check that an attribute satisfies the constraints
    def verify(self, attr: Attribute, constraint_vars: dict[str, Attribute]) -> None:
        # We first check that the attribute is an ArrayAttr
        if not isa(attr, ArrayAttr[Attribute]):
            raise VerifyException(f"expected attribute ArrayData but got {attr}")

        # We check the constraint for all elements in the array
        for e in attr.data:
            self.elem_constr.verify(e, constraint_vars)


array_constraint = ArrayOfConstraint(IntAttr)

# This will pass without triggering an exception
array_constraint.verify(ArrayAttr([IntAttr(42)]), {})
array_constraint.verify(ArrayAttr([IntAttr(3), IntAttr(7)]), {})

In [14]:
# This will trigger an exception, since the attribute is not an array
try:
    array_constraint.verify(i32, {})
except Exception as e:
    print(e)

expected data ArrayData but got i32


In [15]:
# This will trigger an exception, since the array contains attribute that do not satisfies the constraint
try:
    array_constraint.verify(ArrayAttr([IntAttr(42), StringAttr("ga")]), {})
except Exception as e:
    print(e)

"ga" should be of base attribute int


## Attribute definition

### Data attributes

`Data` attribute types are defined by inheriting the python `Data` class. Each data attribute definition should define a `name`, and two methods for conversion to a string representation. Here is for example the definition of `IntAttr`:

In [16]:
from xdsl.irdl import irdl_attr_definition
from xdsl.ir import Data
from xdsl.parser import Parser


@irdl_attr_definition
class MyIntAttr(Data[int]):
    name = "my_int"

    @staticmethod
    def parse_parameter(parser: Parser) -> int:
        data = parser.parse_integer()
        return data

    def print_parameter(self, printer: Printer) -> None:
        printer.print_string(f"{self.data}")


MyIntAttr(3).print_parameter(printer)

3

### Parametrized attributes

`ParametrizedAttribute` attribute types are defined using the `irdl_attr_definition` decorator on a class. Such class should contain a `name` field specifying the attribute name. Parameters are added to attribute definitions by defining fields containing a `ParameterDef`. The field names correspond to the parameter names, and `ParameterDef` contains a constraint that should be respected by this parameter. The order of the fields correspond to the order of the parameters when using the attribute. Upon construction of an attribute, all constraints will be checked, and an exception will be raised if the invariants are not satisfied.

Here is an example of an integer type definition:

In [17]:
from xdsl.ir import ParametrizedAttribute
from xdsl.irdl import ParameterDef


# Represent an integer type with a given bitwidth
@irdl_attr_definition
class MyIntegerType(ParametrizedAttribute):
    # Name of the type. This is used for printing and parsing.
    name = "integer_type"

    # Only parameter of the type, with an `EqAttrConstraint` constraint.
    # Note the use of the attribute constraint coercion.
    width: ParameterDef[IntAttr]


my_i32 = MyIntegerType([IntAttr(32)])

# This will trigger an exception, since the attribute only expect a single attribute
try:
    MyIntegerType([IntAttr(32), IntAttr(64)])
except Exception as e:
    print(e)

# This will trigger an exception, since the attribute is not an IntAttr
try:
    MyIntegerType([StringAttr("ga")])
except Exception as e:
    print(e)

In integer_type attribute verifier: 1 parameters expected, got 2
"ga" should be of base attribute int


Each parameter can be accessed using the `parameters` field.

In [18]:
my_i32.parameters

[IntAttr(data=32)]

Our attribute definition also defines accessors for each parameter based on the name given in the `ParameterDef` field:

In [19]:
my_i32.width

IntAttr(data=32)

## Operation definition

Operations are defined similarly to `ParametrizedAttribute`, by using the `irdl_op_definition` decorator. The decorator allows the definition of expected operands, results, attributes, and regions. Each definition should contain a `name` static field, which is used for parsing and printing:

In [20]:
from xdsl.irdl import irdl_op_definition, IRDLOperation


@irdl_op_definition
class MyEmptyOp(IRDLOperation):
    name = "my_dialect.my_op"


my_op = MyEmptyOp.build()
printer.print_op(my_op)

"my_dialect.my_op"() : () -> ()

### Operands and Results

Operands and results are added using fields containing `Operand` and `OpResult`, which each contain an attribute constraint. The order correspond to the operand and result order, and the constraint applies on the SSA variable type.

Here is an example of an operation defining operands and a result:

In [21]:
from typing import Annotated

from xdsl.ir import OpResult
from xdsl.irdl import Operand, operand_def, result_def
from xdsl.dialects.arith import Constant


@irdl_op_definition
class Addi32(IRDLOperation):
    name = "arith.addi32"

    # Definition of operands and results.
    # Note the use of the attribute constraint coercion.
    input1: Operand = operand_def(i32)
    input2: Operand = operand_def(i32)
    output: OpResult = result_def(i32)


i32_ssa_var = Constant.from_attr(IntegerAttr.from_int_and_width(62, 32), i32)
my_addi32 = Addi32.build(
    operands=[i32_ssa_var.result, i32_ssa_var.result], result_types=[i32]
)
printer.print_op(i32_ssa_var)
print()
printer.print_op(my_addi32)

%0 = arith.constant 62 : i32
%1 = "arith.addi32"(%0, %0) : (i32, i32) -> i32

The operation expects exactly the number of operands and results defined and checks that operands and results satisfy their invariants using the `verify` method.

In [22]:
# This will pass since the invariants are satisfied
my_addi32.verify()

In [23]:
# Wrong number of operands
try:
    bad_addi32 = Addi32.build(operands=[i32_ssa_var], result_types=[i32])
except Exception as e:
    print(e)

Expected 2 operand, but got 1


In [24]:
# Wrong number of results
try:
    bad_addi32 = Addi32.build(
        operands=[i32_ssa_var, i32_ssa_var], result_types=[i32, i32]
    )
except Exception as e:
    print(e)

Expected 1 result, but got 2


In [25]:
# Wrong result type
bad_addi32 = Addi32.build(operands=[i32_ssa_var, i32_ssa_var], result_types=[i64])

try:
    bad_addi32.verify()
except Exception as e:
    print(e)

Operation does not verify: arith.addi32 operation does not verify

%0 = "arith.addi32"(%1, %1) : (i32, i32) -> i64



%0 = "arith.addi32"(%1, %1) : (i32, i32) -> i64




As with `ParametrizedAttribute`, each operand and result definition defines accessors to easily access operands and results:

In [26]:
assert my_addi32.input1 == my_addi32.operands[0]
assert my_addi32.input2 == my_addi32.operands[1]
assert my_addi32.output == my_addi32.results[0]

### Constraint variables

Constraint variables can directly be used in Operation and Attribute definitions by using a `TypeAlias` annotated with a `ConstraintVar`. 

In [27]:
from xdsl.irdl import ConstraintVar


@irdl_op_definition
class BinaryOp(IRDLOperation):
    name = "binary_op"

    T = Annotated[IntegerType, ConstraintVar("T")]

    lhs: Operand = operand_def(T)
    rhs: Operand = operand_def(T)
    result: OpResult = result_def(T)


op = BinaryOp.build(operands=[i32_ssa_var, i32_ssa_var], result_types=[i32])
op.verify()

op_incorrect = BinaryOp.build(operands=[i32_ssa_var, i32_ssa_var], result_types=[i64])
try:
    op_incorrect.verify()
except Exception as e:
    print(e)

Operation does not verify: binary_op operation does not verify

%0 = "binary_op"(%1, %1) : (i32, i32) -> i64



%0 = "binary_op"(%1, %1) : (i32, i32) -> i64




### Variadic operands and results

Operand and result definitions can be defined variadic, meaning that their definition can have different numbers of operands or results. Variadic definitions are defined with `VarOperand` and `VarResultDef`.

In [28]:
from xdsl.irdl import VarOperand, var_operand_def


@irdl_op_definition
class AddVariadicOp(IRDLOperation):
    name = "add_variadic"
    ops: VarOperand = var_operand_def(i32)
    res: OpResult = result_def(i32)


i32_ssa_var = Constant.from_attr(IntegerAttr.from_int_and_width(62, 32), i32)
add_op = AddVariadicOp.build(operands=[[i32_ssa_var] * 3], result_types=[i32])
printer.print_op(i32_ssa_var)
print()
printer.print_op(add_op)

%2 = arith.constant 62 : i32
%3 = "add_variadic"(%2, %2, %2) : (i32, i32, i32) -> i32

Generated accessors return list of SSA variables instead of a single variable:

In [29]:
print(len(add_op.ops))

3


Since it may be ambiguous, it is not possible to simply define two variadic operands, or two variadic results. To do so, the operation definition requires the `AttrSizedOperandSegments` or `AttrSizedResultSegments` IRDL option, which require the operation to contain a `operand_segment_sizes` or `result_segment_sizes` attribute, containing the size of the variadic operands, and variadic results.

In [30]:
from xdsl.irdl import AttrSizedOperandSegments
from xdsl.dialects.builtin import VectorType


@irdl_op_definition
class AddVariadicOp2(IRDLOperation):
    name = "add_variadic"
    ops1: VarOperand = var_operand_def(i32)
    ops2: VarOperand = var_operand_def(i32)
    res: OpResult = result_def(i32)

    irdl_options = [AttrSizedOperandSegments()]


i32_ssa_var = Constant.from_attr(IntegerAttr.from_int_and_width(62, 32), i32)
add_op2 = AddVariadicOp2.build(
    operands=[[i32_ssa_var] * 2, [i32_ssa_var]],
    result_types=[i32],
    attributes={
        "operand_segment_sizes": VectorType.from_element_type_and_shape(i32, [2, 1])
    },
)
print("Length of add_op2.ops1:", len(add_op2.ops1))
print("Length of add_op2.ops2:", len(add_op2.ops2))

Length of add_op2.ops1: 2
Length of add_op2.ops2: 1


In addition to variadic operands and results, IRDL also allows the definition of optional operands and results. Optional operands and results are essentially variadic operands and regions that are constrained to have either 0 or 1 elements. They are defined using `OptOperand` and `OptResultDef`, and define accessors returning optional SSA variables.

In [31]:
from xdsl.irdl import OptOperand, opt_operand_def


@irdl_op_definition
class AddVariadicOp2(IRDLOperation):
    name = "add_optional"
    ops1: Operand = operand_def(i32)
    ops2: OptOperand = opt_operand_def(i32)
    res: OpResult = result_def(i32)


i32_ssa_var = Constant.from_attr(IntegerAttr.from_int_and_width(62, 32), i32)
add_op3 = AddVariadicOp2.build(
    operands=[i32_ssa_var, [i32_ssa_var]], result_types=[i32]
)
print(add_op3.ops2)

add_op4 = AddVariadicOp2.build(operands=[i32_ssa_var, []], result_types=[i32])
print(add_op4.ops2)

<OpResult[i32] index: 0, operation: arith.constant, uses: 2>
None


### Attributes definition

Attribute definitions are defined using `OpAttr`. The field name correspond to the expected attribute name.

In [32]:
from xdsl.irdl import attr_def


@irdl_op_definition
class StringAttrOp(IRDLOperation):
    name = "string_attr_op"
    value: StringAttr = attr_def(StringAttr)


my_attr_op = StringAttrOp.build(attributes={"value": StringAttr("ga")})
my_attr_op.verify()
printer.print_op(my_attr_op)

"string_attr_op"() {"value" = "ga"} : () -> ()

The defined verifier ensures that the attribute is present:

In [33]:
my_attr_op = StringAttrOp.build()
try:
    my_attr_op.verify()
except Exception as e:
    print(e)

Operation does not verify: attribute value expected

"string_attr_op"() : () -> ()




And expects that the attribute respect the constraint:

In [34]:
try:
    my_attr_op = StringAttrOp.build(attributes={"value": IntAttr(42)})
except Exception as e:
    print(e)

Note that contrary to operands and results, other attributes may appear in an operation, even though they are not defined by the operation:


In [35]:
my_attr_op = StringAttrOp.build(
    attributes={"value": StringAttr("ga"), "other_attr": IntAttr(42)}
)
my_attr_op.verify()
printer.print_op(my_attr_op)

"string_attr_op"() {"value" = "ga", "other_attr" = #int<42>} : () -> ()

### Regions

Regions definitions are defined using `Region` or `SingleBlockRegion` annotations. The second definition constrains the region to contain a single block, and both definitions allows to further constraint the region by giving a constraint for the entry basic block parameters.

In [36]:
from xdsl.irdl import Region, Block, region_def
from xdsl.traits import NoTerminator


@irdl_op_definition
class WhileOp(IRDLOperation):
    name = "while_op"
    value: Region = region_def()
    traits = frozenset([NoTerminator()])


region = Region(Block(arg_types=[i32]))
region_op = WhileOp.build(regions=[region])
region_op.verify()
printer.print_op(region_op)

"while_op"() ({
^0(%4 : i32):
}) : () -> ()

### Additional verifiers

`irdl_op_definition` is not expressive enough to define arbitrary constraints, especially for constraints spanning over multiple operand and result definitions. To circumvent that, definitions may define a `verify_` method that will be called in the generated verifier:

In [37]:
from xdsl.dialects.builtin import IntegerType
from xdsl.dialects.arith import Addi


@irdl_op_definition
class MyAddi(IRDLOperation):
    name = "std.addi"
    input1: Operand = operand_def(IntegerType)
    input2: Operand = operand_def(IntegerType)
    output: OpResult = result_def(IntegerType)

    # Ensure that the inputs and outputs have the same type:
    def verify_(self) -> None:
        if self.input1.type != self.input2.type or self.input2.type != self.output.type:
            raise Exception("expect all input and output types to be equal")


i32_ssa_var = Constant.from_attr(IntegerAttr.from_int_and_width(62, 32), i32)
add_op = Addi.build(operands=[i32_ssa_var, i32_ssa_var], result_types=[i32])
# This will pass, since all operands and results have the same type
add_op.verify()

# This will raise an error, since the result has a different type than the operands
bad_add_op = Addi.build(operands=[i32_ssa_var, i32_ssa_var], result_types=[i64])
try:
    bad_add_op.verify()
except Exception as e:
    print(e)

Operation does not verify: arith.addi operation does not verify

%0 = "arith.addi"(%1, %1) : (i32, i32) -> i64



%0 = "arith.addi"(%1, %1) : (i32, i32) -> i64


